# فیلترهای خطی / Linear Filters

**هدف:** یادگیری و پیاده‌سازی فیلترهای خطی برای حذف نویز از تصاویر

---

## محتوا:
1. میانگین‌گیری از چند تصویر
2. فیلتر میانگین محلی
3. فیلتر گاوسی
4. مقایسه روش‌ها

In [ ]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

# تنظیمات نمایش
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. توابع کمکی / Helper Functions

In [ ]:
def add_gaussian_noise(image: np.ndarray, mean: float = 0, sigma: float = 20) -> np.ndarray:
    """
    اضافه کردن نویز گاوسی به تصویر
    
    Args:
        image: تصویر ورودی
        mean: میانگین نویز (معمولاً 0)
        sigma: انحراف معیار نویز
    
    Returns:
        تصویر نویزی
    """
    # تبدیل به int16 برای جلوگیری از overflow
    image_int16 = image.astype(np.int16)
    
    # تولید نویز گاوسی
    noise = np.random.normal(mean, sigma, image.shape).astype(np.int16)
    
    # اضافه کردن نویز
    noisy_image = image_int16 + noise
    
    # محدود کردن به بازه [0, 255]
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    
    return noisy_image


def calculate_snr(original: np.ndarray, noisy: np.ndarray) -> float:
    """
    محاسبه نسبت سیگنال به نویز (SNR)
    
    Args:
        original: تصویر اصلی
        noisy: تصویر نویزی
    
    Returns:
        SNR به دسی‌بل
    """
    signal_power = np.sum(original.astype(np.float64) ** 2)
    noise = (noisy.astype(np.float64) - original.astype(np.float64))
    noise_power = np.sum(noise ** 2)
    
    if noise_power == 0:
        return float('inf')
    
    snr = 10 * np.log10(signal_power / noise_power)
    return snr


def calculate_psnr(original: np.ndarray, processed: np.ndarray) -> float:
    """
    محاسبه PSNR (Peak Signal-to-Noise Ratio)
    
    Args:
        original: تصویر اصلی
        processed: تصویر پردازش شده
    
    Returns:
        PSNR به دسی‌بل
    """
    mse = np.mean((original.astype(np.float64) - processed.astype(np.float64)) ** 2)
    
    if mse == 0:
        return float('inf')
    
    psnr = 10 * np.log10(255**2 / mse)
    return psnr


def display_images(images: list, titles: list, cmap='gray'):
    """
    نمایش چند تصویر در کنار هم
    """
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))
    
    if n == 1:
        axes = [axes]
    
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap=cmap)
        ax.set_title(title, fontsize=14, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

print("✓ توابع کمکی آماده شدند")

## 2. ایجاد تصویر نمونه / Create Sample Image

In [ ]:
# ایجاد یک تصویر ساده برای تست
# می‌توانید تصویر خود را بارگذاری کنید

# روش 1: ایجاد تصویر مصنوعی
height, width = 256, 256
original_image = np.zeros((height, width), dtype=np.uint8)

# اضافه کردن اشکال مختلف
cv2.rectangle(original_image, (50, 50), (150, 150), 200, -1)
cv2.circle(original_image, (200, 200), 40, 150, -1)
cv2.line(original_image, (0, 0), (255, 255), 100, 2)

# روش 2: بارگذاری تصویر واقعی (اگر دارید)
# original_image = cv2.imread('your_image.jpg', cv2.IMREAD_GRAYSCALE)

display_images([original_image], ['تصویر اصلی / Original Image'])
print(f"اندازه تصویر: {original_image.shape}")

## 3. اضافه کردن نویز / Add Noise

In [ ]:
# اضافه کردن نویز گاوسی با انحراف معیار مختلف
sigma_values = [10, 20, 30]
noisy_images = []

for sigma in sigma_values:
    noisy = add_gaussian_noise(original_image, mean=0, sigma=sigma)
    noisy_images.append(noisy)
    snr = calculate_snr(original_image, noisy)
    print(f"σ = {sigma}: SNR = {snr:.2f} dB")

# نمایش تصاویر نویزی
titles = [f'σ = {s}' for s in sigma_values]
display_images(noisy_images, titles)

## 4. روش اول: میانگین‌گیری از چند تصویر
## Method 1: Image Averaging

**تئوری:** اگر n تصویر از یک صحنه ثابت داشته باشیم، میانگین‌گیری از آنها نویز را با ضریب √n کاهش می‌دهد.

**فرمول:**
```
f_avg(i,j) = (1/n) Σ f_k(i,j)
```

In [ ]:
# تولید 8 تصویر نویزی از یک صحنه
n_images = 8
sigma = 30

noisy_set = [add_gaussian_noise(original_image, sigma=sigma) for _ in range(n_images)]

# محاسبه میانگین
averaged_image = np.mean(noisy_set, axis=0).astype(np.uint8)

# محاسبه معیارها
snr_single = calculate_snr(original_image, noisy_set[0])
snr_averaged = calculate_snr(original_image, averaged_image)
psnr_averaged = calculate_psnr(original_image, averaged_image)

print(f"SNR تک تصویر: {snr_single:.2f} dB")
print(f"SNR میانگین {n_images} تصویر: {snr_averaged:.2f} dB")
print(f"PSNR میانگین: {psnr_averaged:.2f} dB")
print(f"بهبود: {snr_averaged - snr_single:.2f} dB")

# نمایش نتایج
display_images(
    [original_image, noisy_set[0], averaged_image],
    ['اصلی', f'نویزی (σ={sigma})', f'میانگین {n_images} تصویر']
)

## 5. روش دوم: فیلتر میانگین محلی
## Method 2: Local Average Filter

**تئوری:** محاسبه میانگین در یک پنجره محلی (مثلاً 3×3) اطراف هر پیکسل.

**ماسک 3×3:**
```
    1/9 × [1 1 1]
          [1 1 1]
          [1 1 1]
```

In [ ]:
# استفاده از تصویر نویزی
noisy_image = add_gaussian_noise(original_image, sigma=20)

# فیلتر میانگین با اندازه‌های مختلف
kernel_sizes = [3, 5, 7]
filtered_images = []

for ksize in kernel_sizes:
    filtered = cv2.blur(noisy_image, (ksize, ksize))
    filtered_images.append(filtered)
    
    snr = calculate_snr(original_image, filtered)
    psnr = calculate_psnr(original_image, filtered)
    print(f"Kernel {ksize}×{ksize}: SNR = {snr:.2f} dB, PSNR = {psnr:.2f} dB")

# نمایش نتایج
all_images = [original_image, noisy_image] + filtered_images
all_titles = ['اصلی', 'نویزی'] + [f'فیلتر {k}×{k}' for k in kernel_sizes]
display_images(all_images, all_titles)

## 6. روش سوم: فیلتر گاوسی
## Method 3: Gaussian Filter

**تئوری:** استفاده از وزن‌های گاوسی به جای وزن‌های یکسان. پیکسل‌های نزدیک‌تر وزن بیشتری دارند.

**تابع گاوسی:**
```
G(x,y) = (1/2πσ²) × exp(-(x²+y²)/2σ²)
```

In [ ]:
# فیلتر گاوسی با σ مختلف
sigma_values = [0.5, 1.0, 2.0]
gaussian_filtered = []

for sigma in sigma_values:
    # اندازه kernel باید فرد باشد
    ksize = int(6 * sigma + 1)
    if ksize % 2 == 0:
        ksize += 1
    
    filtered = cv2.GaussianBlur(noisy_image, (ksize, ksize), sigma)
    gaussian_filtered.append(filtered)
    
    snr = calculate_snr(original_image, filtered)
    psnr = calculate_psnr(original_image, filtered)
    print(f"σ = {sigma}: SNR = {snr:.2f} dB, PSNR = {psnr:.2f} dB")

# نمایش نتایج
all_images = [original_image, noisy_image] + gaussian_filtered
all_titles = ['اصلی', 'نویزی'] + [f'Gaussian σ={s}' for s in sigma_values]
display_images(all_images, all_titles)

## 7. مقایسه روش‌ها / Comparison

بیایید همه روش‌ها را با هم مقایسه کنیم.

In [ ]:
# اعمال همه روش‌ها
noisy = add_gaussian_noise(original_image, sigma=25)

# روش 1: میانگین 8 تصویر
noisy_set = [add_gaussian_noise(original_image, sigma=25) for _ in range(8)]
method1 = np.mean(noisy_set, axis=0).astype(np.uint8)

# روش 2: فیلتر میانگین 5×5
method2 = cv2.blur(noisy, (5, 5))

# روش 3: فیلتر گاوسی σ=1.5
method3 = cv2.GaussianBlur(noisy, (7, 7), 1.5)

# محاسبه معیارها
results = [
    ('نویزی', noisy),
    ('میانگین 8 تصویر', method1),
    ('فیلتر میانگین 5×5', method2),
    ('فیلتر گاوسی σ=1.5', method3)
]

print("\n" + "="*60)
print("مقایسه روش‌ها:")
print("="*60)
for name, img in results:
    snr = calculate_snr(original_image, img)
    psnr = calculate_psnr(original_image, img)
    print(f"{name:25s} | SNR: {snr:6.2f} dB | PSNR: {psnr:6.2f} dB")

# نمایش بصری
images = [original_image] + [img for _, img in results]
titles = ['اصلی'] + [name for name, _ in results]
display_images(images, titles)

## 8. تمرین عملی / Practical Exercise

**وظیفه:** تصویر خود را بارگذاری کنید و:
1. نویز گاوسی با σ=30 اضافه کنید
2. سه روش فیلترینگ را اعمال کنید
3. بهترین روش را بر اساس PSNR انتخاب کنید

In [ ]:
# کد تمرین شما اینجا
# TODO: بارگذاری تصویر
# TODO: اضافه کردن نویز
# TODO: اعمال فیلترها
# TODO: مقایسه نتایج

pass

## نتیجه‌گیری / Conclusion

**نکات کلیدی:**

1. **میانگین‌گیری از چند تصویر:**
   - ✓ بهترین نتیجه برای نویز گاوسی
   - ✗ نیاز به چند تصویر از یک صحنه
   - ✗ صحنه باید ثابت باشد

2. **فیلتر میانگین محلی:**
   - ✓ ساده و سریع
   - ✓ فقط یک تصویر نیاز دارد
   - ✗ لبه‌ها را محو می‌کند

3. **فیلتر گاوسی:**
   - ✓ لبه‌ها را بهتر حفظ می‌کند
   - ✓ کنترل بیشتر با پارامتر σ
   - ✗ کمی پیچیده‌تر

**توصیه:** برای نویز گاوسی، فیلتر گاوسی معمولاً بهترین انتخاب است.

---

**بعدی:** `03_nonlinear_filters.ipynb` - فیلترهای غیرخطی برای نویز نمک و فلفل